In [277]:
import os
from pandas_datareader import data as pdr
import pandas as pd
import numpy as np
import yfinance as yf
from tqdm import tqdm
import requests
import urllib.request
import re
import time
import tensorflow as tf
import tensorflow.keras as keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from IPython.display import display, HTML
import csv
import warnings


In [278]:
yf.pdr_override()
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)
START_DATE = "1984-12-31"
END_DATE = "2021-12-31"

In [279]:
#Utility function to reformat yahoo finance numbers
def data_string_to_float(number_string):
    if ("N/A" in number_string) or ("NaN" in number_string):
        return "N/A"
    elif number_string == ">0":
        return 0
    elif "B" in number_string:
        return float(number_string.replace("B", "")) * 1000000000
    elif "M" in number_string:
        return float(number_string.replace("M", "")) * 1000000
    elif "K" in number_string:
        return float(number_string.replace("K", "")) * 1000
    else:
        return float(number_string)

In [280]:
#Get list of tickers
def file_to_list(file):
    rtn: object = []
    file_object: object = open(file, "r")
    rtn: object = file_object.read().splitlines()
    file_object.close()
    return list(filter(None, pd.unique(rtn).tolist())) # Remove Empty/Duplicates Values
    pass

# Example #    
ticker_list: object = file_to_list('/Users/nickd/Downloads/constituents_symbols.txt') 
all_data = pdr.get_data_yahoo(ticker_list, START_DATE, END_DATE)
stock_data = all_data["Adj Close"]

[*********************100%***********************]  503 of 503 completed

3 Failed downloads:
- CEG: Data doesn't exist for startDate = 473320800, endDate = 1640930400
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted


In [281]:
stock_data.dropna(how="all", axis=1, inplace=True)
missing_tickers = [
    ticker for ticker in ticker_list if ticker.upper() not in stock_data.columns
]
print(f"{len(missing_tickers)} tickers are missing: \n {missing_tickers} ")
stock_data.ffill(inplace=True)
stock_data.to_csv("stock_prices.csv")

/var/folders/wm/vl1dqz054m7gmq9rpgw2b_vm0000gn/T/ipykernel_33915/4029096740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.dropna(how="all", axis=1, inplace=True)
/var/folders/wm/vl1dqz054m7gmq9rpgw2b_vm0000gn/T/ipykernel_33915/4029096740.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data.ffill(inplace=True)


3 tickers are missing: 
 ['BRK.B', 'BF.B', 'CEG'] 


In [282]:
index_data = yf.download (tickers = "^GSPC", start = START_DATE, 
                              end = END_DATE, interval = "1d")
index_data.to_csv("sp500_index.csv")

[*********************100%***********************]  1 of 1 completed


In [283]:
df = pd.DataFrame()

for ticker in ticker_list:
    ticker = ticker.upper()

    stock_ohlc = pdr.get_data_yahoo(ticker, START_DATE, END_DATE)
    if stock_ohlc.empty:
        print(f"No data for {ticker}")
        continue
    adj_close = stock_ohlc["Adj Close"].rename(ticker)
    df = pd.concat([df, adj_close], axis=1)
df.to_csv("stock_prices.csv")

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [285]:
sp500_raw_data = pd.read_csv("sp500_index.csv", index_col="Date", parse_dates=True)
stock_raw_data = pd.read_csv("stock_prices.csv", index_col="Date", parse_dates=True)

In [286]:
sp500_raw_data.ffill(inplace=True)
stock_raw_data.ffill(inplace=True)
stock_list = [x[0] for x in ticker_list]
stock_list = stock_list[1:]

In [293]:
def create_stock_df(ticker):
     # Creating a new dataframe which we will later fill.
    path = "/Users/nickd/Desktop/Financial CSV's/" + ticker + "_annual_financials.csv"
    tickers = []
    unix = []
    price = []
    stockChange = []
    sp500 = []
    sp500Change = []
    add_df = pd.DataFrame()
    try:
        with open(path, 'r') as f:          # Read lines separately
            reader = csv.reader(f)
            for i, line in enumerate(reader):
                new_df_row = line
                newdf = pd.DataFrame(new_df_row)
                add_df = pd.concat([add_df, newdf], axis=1)
    except FileNotFoundError:
        print("error")
    add_df.columns = add_df.iloc[0]
    add_df = add_df.iloc[4: , :]
    add_df = add_df.rename(columns={"name": "Date"})
    add_df = add_df[['Date', 'TotalRevenue', "NetIncomeCommonStockholders", "BasicEPS", "DilutedEPS", "TotalExpenses", "EBIT", "PretaxIncome"]]
    for i in range(add_df.shape[0]):
        tickers.append(ticker)
        date_time = add_df.at[add_df.index[i],'Date']
        format_data = "%m/%d/%Y"
        date = datetime.strptime(date_time, format_data)
        unix_time = time.mktime(date.timetuple())
        unix.append(unix_time)
        current_date = datetime.fromtimestamp(unix_time).strftime("%Y-%m-%d")
        one_year_later = datetime.fromtimestamp(unix_time + 31536000).strftime(
                "%Y-%m-%d"
        )
        try:
        # SP500 prices now and one year later, and the percentage change
            sp500_price = float(sp500_raw_data.loc[current_date, "Adj Close"])
            sp500_1y_price = float(sp500_raw_data.loc[one_year_later, "Adj Close"])
            sp500_p_change = round(
                ((sp500_1y_price - sp500_price) / sp500_price * 100), 2
            )
            sp500Change.append(sp500_p_change)
        except:
            current_date = datetime.fromtimestamp(unix_time - 259200).strftime("%Y-%m-%d")
            one_year_later = datetime.fromtimestamp(unix_time + 31536000 - 259200).strftime(
                "%Y-%m-%d")
            sp500_price = float(sp500_raw_data.loc[current_date, "Adj Close"])
            sp500_1y_price = float(sp500_raw_data.loc[one_year_later, "Adj Close"])
            sp500_p_change = round(
                ((sp500_1y_price - sp500_price) / sp500_price * 100), 2
            )
            sp500Change.append(sp500_p_change)
          # Stock prices now and one year later. We need a try/except because some data is missing
        stock_price, stock_1y_price = "N/A", "N/A"
        try:
            stock_price = float(stock_raw_data.loc[current_date, ticker.upper()])
            stock_1y_price = float(stock_raw_data.loc[one_year_later, ticker.upper()])
        except KeyError:
            print("Price retrieval error for " + ticker + " at " + current_date)
            continue

        stock_p_change = round(
            ((stock_1y_price - stock_price) / stock_price * 100), 2
        )
        stockChange.append(stock_p_change)
        price.append(stock_price)
        sp500.append(sp500_price)

    add_df['Ticker'] = tickers
    add_df['Unix'] = unix
    add_df['SP500Change'] = sp500Change
    add_df['StockChange'] = stockChange
    add_df['Stock Price'] = price
    add_df['SP500 Price'] = sp500
    return add_df

In [301]:
path = "/Users/nickd/Desktop/Financial CSV's/"
df = pd.DataFrame()
numError = 0
for ticker in ticker_list:
    try:
        add = create_stock_df(ticker)
        df = pd.concat([df, add], axis=0)
    except KeyError:
        numError += 1
        continue
    except IndexError:
        print(ticker + " Empty")
        continue
    except ValueError:
        print(ticker + " Value Error")
        continue
print(numError)
df.dropna(axis=0, subset=["Stock Price", "StockChange"], inplace=True)
df.to_csv("keystats.csv", index=False)

error
BF.B Empty
error
CPB Empty
Price retrieval error for CEG at 2019-12-31
Price retrieval error for CEG at 2018-12-31
CEG Value Error
69


In [11]:
def status_calc(stock, sp500, outperformance=10):
    """A simple function to classify whether a stock outperformed the S&P500
    :param stock: stock price
    :param sp500: S&P500 price
    :param outperformance: stock is classified 1 if stock price > S&P500 price + outperformance
    :return: true/false
    """
    if outperformance < 0:
        raise ValueError("outperformance must be positive")
    return stock - sp500 >= outperformance

In [302]:
data_df = pd.read_csv("keystats.csv")
data_df.drop_duplicates()
data_df.dropna(inplace=True)
data_df = data_df.reindex(np.random.permutation(data_df.index))
features = data_df.columns[4:]
X = np.array(data_df[features].values)
sc = MinMaxScaler(feature_range=(0,1))
X = sc.fit_transform(X)
y = np.array(
    status_calc(
        data_df["StockChange"], data_df["SP500Change"], outperformance=10
    )
)
y.astype(int)
X = np.reshape(X, (X.shape[0], X.shape[1], 1))
model = Sequential()
model.add(LSTM(units=50,return_sequences=True,input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50,return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.fit(X,y,epochs=100,batch_size=32)

ValueError: could not convert string to float: '8,224,000,000'

In [15]:
display(data_df)

,Date,Unix,Ticker,Price,stock_p_change,SP500,SP500_p_change,Total Debt/Equity,Trailing P/E,Price/Book,Profit Margin,Operating Margin,Return on Assets,Return on Equity,Revenue Per Share,Market Cap,Enterprise Value,Forward P/E,PEG Ratio,Enterprise Value/Revenue,Enterprise Value/EBITDA,Revenue,Gross Profit,EBITDA,Net Income Avl to Common,Diluted EPS,Qtrly Earnings Growth,Qtrly Revenue Growth,Total Cash,Total Cash Per Share,Total Debt,Current Ratio,Book Value Per Share,Cash Flow,Beta,% Held by Insiders,% Held by Institutions,Shares Short (as of,Short Ratio,Short % of Float,Shares Short (prior,Current Price,Shares Outstanding
